<a href="https://colab.research.google.com/github/VesleAnne/Diploma/blob/main/Notebooks/Text_preprocessing_and_enbeddings2_e5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Perform Google Colab installs
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch -q # requires torch 2.1.1+ (for efficient sdpa implementation)
#    !pip install PyMuPDF # for reading PDFs with Python if needed
    !pip install python-docx -q #for reading Word with Python
    !pip install tqdm -q # for progress bars
    !pip install sentence-transformers -q # for embedding models
    !pip install accelerate -q # for quantization model loading
    !pip install bitsandbytes -q # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation -q # for faster attention mechanism = faster LLM inference

[INFO] Running in Google Colab, installing requirements.
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 779.1/779.1 MB 945.6 kB/s eta 0:00:00
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 176.2/176.2 MB 2.4 MB/s eta 0:00:00
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 168.1/168.1 MB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
  

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from docx import Document
import random
import pandas as pd
import torch
from tqdm.auto import tqdm
import re
import numpy as np
from sentence_transformers import util, SentenceTransformer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%cd '/content/drive/MyDrive/–ú–§–¢–ò/–≤–∫—Ä/RAG'

[Errno 2] No such file or directory: '/content/drive/MyDrive/–ú–§–¢–ò/–≤–∫—Ä/RAG'
/content


In [ ]:
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()  # Replace newline characters with spaces and strip leading/trailing whitespace
    return cleaned_text

In [ ]:
def open_and_read_word(docx_path: str) -> list[dict]:
    """
    Opens a Word document, reads its content page by page, and collects statistics.

    Parameters:
        docx_path (str): The file path to the Word document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = Document(docx_path)  # Open the Word document using python-docx
    pages_and_texts = []
    for page_number, paragraph in enumerate(doc.paragraphs):
        text = paragraph.text.strip()  # Get text from the paragraph and remove leading/trailing whitespace
        if text:  # Exclude empty paragraphs
            text = text_formatter(text)  # Format the text using the text_formatter function
            pages_and_texts.append({"page_number": page_number,
                                    "page_char_count": len(text),
                                    "page_word_count": len(text.split()),  # Count words by splitting on whitespace
                                    "page_sentence_count_raw": len(text.split(". ")),  # Count sentences by splitting on periods followed by spaces
                                    "page_token_count": len(text) / 4,  # Estimate token count
                                    "text": text})
    return pages_and_texts

In [ ]:
# Open and read the Word document, extracting text and collecting statistics
docx_path = '/content/drive/MyDrive/–ú–§–¢–ò/–≤–∫—Ä/RAG/DATA/greendata.docx'
pages_and_texts = open_and_read_word(docx_path=docx_path)

In [ ]:
random.sample(pages_and_texts, k=3)

[{'page_number': 2217,
  'page_char_count': 367,
  'page_word_count': 42,
  'page_sentence_count_raw': 3,
  'page_token_count': 91.75,
  'text': '–ù–∞–±–æ—Ä –∫–Ω–æ–ø–æ–∫ –ø–∞–Ω–µ–ª–∏ —É–ø—Ä–∞–≤–ª–µ–Ω–∏—è –∑–∞–≤–∏—Å–∏—Ç –æ—Ç –æ—Ç–æ–±—Ä–∞–∂–∞–µ–º–æ–≥–æ –æ–±—ä–µ–∫—Ç–∞ —Ç–∏–ø–∞ –°–∏—Å—Ç–µ–º—ã –∏ —Ñ–æ—Ä–º–∏—Ä—É–µ—Ç—Å—è –∞–≤—Ç–æ–º–∞—Ç–∏—á–µ—Å–∫–∏. –ü—Ä–∏ —ç—Ç–æ–º, –æ—Ç–æ–±—Ä–∞–∂–µ–Ω–∏–µ –¥–æ–ø–æ–ª–Ω–∏—Ç–µ–ª—å–Ω—ã—Ö –∫–Ω–æ–ø–æ–∫ –Ω–∞—Å—Ç—Ä–∞–∏–≤–∞–µ—Ç—Å—è –∞–¥–º–∏–Ω–∏—Å—Ç—Ä–∞—Ç–æ—Ä–æ–º —Å–∞–º–æ—Å—Ç–æ—è—Ç–µ–ª—å–Ω–æ (–Ω–∞–ø—Ä–∏–º–µ—Ä, –æ—Ç–æ–±—Ä–∞–∂–µ–Ω–∏–µ –∫–Ω–æ–ø–∫–∏ –≤—ã–≤–æ–¥–∞ –ø–µ—á–∞—Ç–Ω–æ–π —Ñ–æ—Ä–º—ã –¥–ª—è –æ–ø—Ä–µ–¥–µ–ª–µ–Ω–Ω–æ–≥–æ –æ–±—ä–µ–∫—Ç–∞ –°–∏—Å—Ç–µ–º—ã). –î–æ–ø–æ–ª–Ω–∏—Ç–µ–ª—å–Ω—ã–µ –∫–Ω–æ–ø–∫–∏ –¥–ª—è —Ä–∞–±–æ—Ç—ã —Å –°–∏—Å—Ç–µ–º–æ–π –æ—Ç–æ–±—Ä–∞–∂–∞—é—Ç—Å—è –ø–æ—Å–ª–µ –Ω–∞–∂–∞—Ç–∏—è –∫–Ω–æ–ø–∫–∏ image.'},
 {'page_number': 1381,
  'page_char_count': 87,
  'page_word_count': 12,
  'page_sentence_count_raw': 1,
  'page_token_count': 21.75,
  'text': '–í –æ—Ç–∫—Ä—ã–≤—à–µ–º—Å—

In [ ]:
# Read Excel files
excel1_path = '/content/drive/MyDrive/–ú–§–¢–ò/–≤–∫—Ä/RAG/DATA/faq.xlsx'
#excel2_path = '/content/drive/MyDrive/–ú–§–¢–ò/–≤–∫—Ä/RAG/DATA/–¥–∞–Ω–Ω—ã–µ2.xlsx'

In [ ]:
df_excel1 = pd.read_excel(excel1_path)
df_excel1.head()

–¢–µ–º–∞                                             –í–æ–ø—Ä–æ—Å  \
0  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç                       –ö–∞–∫ –∑–∞—Ä–µ–≥–∏—Å—Ç—Ä–∏—Ä–æ–≤–∞—Ç—å—Å—è –≤ –õ–ö?   
1  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç  –ó–∞—á–µ–º —É–∫–∞–∑—ã–≤–∞—Ç—å —Å—Ç—Ä–∞–Ω—É? |–ó–∞—á–µ–º –≤—ã–±—Ä–∏—Ä–∞—Ç—å —è–∑—ã–∫?...   
2  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç                 –ù–µ –ø—Ä–∏—Ö–æ–¥–∏—Ç –ø–æ–¥—Ç–≤–µ—Ä–∂–¥–µ–Ω–∏–µ –Ω–∞ –ø–æ—á—Ç—É   
3  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç  –ù–µ—Ç –ø–æ—á—Ç—ã/–∑–∞–±—ã–ª –ø–æ—á—Ç—É. –ï—Å—Ç—å –ª–∏ –¥—Ä—É–≥–æ–π –∫–∞–Ω–∞–ª –¥–ª...   
4  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç    –ü–æ—á–µ–º—É —É –º–µ–Ω—è –≤—Å–µ –æ—Ç–æ–±—Ä–∞–∂–∞–µ—Ç—Å—è –Ω–µ –Ω–∞ —Ç–æ–º —è–∑—ã–∫–µ?   

                                        –°—Ö–µ–º–∞ –æ—Ç–≤–µ—Ç–∞ –ö–Ω–æ–ø–∫–∞ —Å—Å—ã–ª–∫–∏ –Ω–∞ –≤–∏–∫–∏   \
0  1. –î–ª—è —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏–∏ –≤ –ª–∏—á–Ω–æ–º –∫–∞–±–∏–Ω–µ—Ç–µ –ø–µ—Ä–µ–π–¥–∏—Ç–µ...       –†–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è –≤ –õ–ö   
1  –ö–Ω–æ–ø–∫–∞ —Å —Ñ–ª–∞–≥–æ–º –≤ –ø—Ä–∞–≤–æ–º –≤–µ—Ä—Ö–Ω–µ–º —É–≥–ª—É –Ω—É–∂–Ω–∞ –¥–ª...                    NaN   
2  I. –ï—Å–ª–∏ –í–∞–º –Ω–µ –ø—Ä–∏—Ö–æ–¥–∏—Ç –ø–∏—Å—å–º–æ –¥–ª—è –ø–æ–¥—Ç–≤–µ—Ä–∂–¥–µ–Ω...                    NaN   
3  –í –Ω–∞—Å—Ç–æ—è—â–µ–µ –≤—Ä–µ–º—è –¥–æ—Å—Ç—É–ø–Ω–∞ —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è —Ç–æ–ª—å–∫–æ ...                    NaN   
4  –í—ã–±–æ—Ä —è–∑—ã–∫–∞ –∑–∞–≤–∏—Å–∏—Ç –æ—Ç –Ω–∞—Å—Ç—Ä–æ–µ–∫ –±—Ä–∞—É–∑–µ—Ä–∞. \nCh...                    NaN   

                                      –°—Å—ã–ª–∫–∞ –Ω–∞ wiki  \
0  https://wiki.greendata.store/%D0%A0%D0%B5%D0%B...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                     –¢—ç–≥–∏ –¥–ª—è –ø–æ–∏—Å–∫–∞  
0  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç, —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è, —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è –õ–ö SA...  
1  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç, —è–∑—ã–∫, –Ω–µ —Ç–æ—Ç —è–∑—ã–∫, –Ω–∞—Å—Ç—Ä–æ–π–∫–∞ —è...  
2  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç, —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è, –ø–æ–¥—Ç–≤–µ—Ä–∂–¥–µ–Ω–∏–µ —Ä–µ–≥...  
3  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç, —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è, —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è –õ–ö SA...  
4  –õ–∏—á–Ω—ã–π –∫–∞–±–∏–Ω–µ—Ç, —è–∑—ã–∫, –Ω–µ —Ç–æ—Ç —è–∑—ã–∫, –Ω–∞—Å—Ç—Ä–æ–π–∫–∞ —è...

In [ ]:
#df_excel2 = pd.read_excel(excel2_path).dropna()
#df_excel2.head()

In [ ]:
text_excel1 = [text_formatter(str(text)) for text in df_excel1['–°—Ö–µ–º–∞ –æ—Ç–≤–µ—Ç–∞']]
#text_excel2 = [text_formatter(str(text)) for text in df_excel2['text']]

In [ ]:
#all_text = text_excel1 + text_excel2

In [ ]:
#def remove_smileys_and_links(text):
#    text = re.sub(r':-?[)DPO()]|;-?[/|]', '', text)
#    text = re.sub(r'jivo media_guid_[\w-]+', '', text)
#    text = re.sub(r'üñº', '', text)
#    text = re.sub(r'üëç', '', text)
#    text = re.sub(r'üê≥', '', text)
#    text = re.sub(r'üëã', '', text)
#    return text

#cleaned_all_text = [remove_smileys_and_links(text) for text in all_text]
#cleaned_all_text = list(filter(lambda x: x.strip(), cleaned_all_text))

In [ ]:
# –î–æ–±–∞–≤–ª—è–µ–º —Ñ—Ä–∞–∑—É "–ü–µ—Ä–µ–∫–ª—é—á–∞—é –Ω–∞ –æ–ø–µ—Ä–∞—Ç–æ—Ä–∞" –≤ –∫–æ–Ω–µ—Ü —Å–ø–∏—Å–∫–∞ 20 —Ä–∞–∑
text_excel1.extend(["–ü–µ—Ä–µ–∫–ª—é—á–∞—é –Ω–∞ –æ–ø–µ—Ä–∞—Ç–æ—Ä–∞"] * 20)

In [ ]:
# –°–æ–∑–¥–∞—ë–º –Ω–æ–≤—ã–π –¥–∞—Ç–∞—Ñ—Ä–µ–π–º —Å —Ç–µ–∫—Å—Ç–æ–º –∏–∑ Excel —Ñ–∞–π–ª–æ–≤
df_new = pd.DataFrame({"text": text_excel1})
df_new.head()

text
0  1. –î–ª—è —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏–∏ –≤ –ª–∏—á–Ω–æ–º –∫–∞–±–∏–Ω–µ—Ç–µ –ø–µ—Ä–µ–π–¥–∏—Ç–µ...
1  –ö–Ω–æ–ø–∫–∞ —Å —Ñ–ª–∞–≥–æ–º –≤ –ø—Ä–∞–≤–æ–º –≤–µ—Ä—Ö–Ω–µ–º —É–≥–ª—É –Ω—É–∂–Ω–∞ –¥–ª...
2  I. –ï—Å–ª–∏ –í–∞–º –Ω–µ –ø—Ä–∏—Ö–æ–¥–∏—Ç –ø–∏—Å—å–º–æ –¥–ª—è –ø–æ–¥—Ç–≤–µ—Ä–∂–¥–µ–Ω...
3  –í –Ω–∞—Å—Ç–æ—è—â–µ–µ –≤—Ä–µ–º—è –¥–æ—Å—Ç—É–ø–Ω–∞ —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è —Ç–æ–ª—å–∫–æ ...
4  –í—ã–±–æ—Ä —è–∑—ã–∫–∞ –∑–∞–≤–∏—Å–∏—Ç –æ—Ç –Ω–∞—Å—Ç—Ä–æ–µ–∫ –±—Ä–∞—É–∑–µ—Ä–∞.  Chr...

In [ ]:
# –î–ª—è –∫–∞–∂–¥–æ–π —Å—Ç—Ä–æ–∫–∏ –≤ –¥–∞—Ç–∞—Ñ—Ä–µ–π–º–µ –≤—ã—á–∏—Å–ª—è–µ–º —Å—Ç–∞—Ç–∏—Å—Ç–∏–∫–∏
for idx, row in df_new.iterrows():
    text = row['text']
    # –ü–æ–¥—Å—á–∏—Ç—ã–≤–∞–µ–º —Å—Ç–∞—Ç–∏—Å—Ç–∏–∫–∏ –¥–ª—è –∫–∞–∂–¥–æ–π —Å—Ç—Ä–æ–∫–∏

    char_count = len(text)
    word_count = len(text.split())
    sentence_count = len(text.split("."))
    token_count = len(text) / 4  # –ü—Ä–∏–º–µ—Ä–Ω–æ–µ –∫–æ–ª–∏—á–µ—Å—Ç–≤–æ —Ç–æ–∫–µ–Ω–æ–≤

    # –ó–∞–ø–∏—Å—ã–≤–∞–µ–º —Å—Ç–∞—Ç–∏—Å—Ç–∏–∫–∏ –æ–±—Ä–∞—Ç–Ω–æ –≤ –¥–∞—Ç–∞—Ñ—Ä–µ–π–º
    df_new.loc[idx, 'page_number'] = 145
    df_new.loc[idx, 'page_char_count'] = char_count
    df_new.loc[idx, 'page_word_count'] = word_count
    df_new.loc[idx, 'page_sentence_count_raw'] = sentence_count
    df_new.loc[idx, 'page_token_count'] = token_count


In [ ]:
df_new.head()

text  page_number  \
0  1. –î–ª—è —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏–∏ –≤ –ª–∏—á–Ω–æ–º –∫–∞–±–∏–Ω–µ—Ç–µ –ø–µ—Ä–µ–π–¥–∏—Ç–µ...        145.0   
1  –ö–Ω–æ–ø–∫–∞ —Å —Ñ–ª–∞–≥–æ–º –≤ –ø—Ä–∞–≤–æ–º –≤–µ—Ä—Ö–Ω–µ–º —É–≥–ª—É –Ω—É–∂–Ω–∞ –¥–ª...        145.0   
2  I. –ï—Å–ª–∏ –í–∞–º –Ω–µ –ø—Ä–∏—Ö–æ–¥–∏—Ç –ø–∏—Å—å–º–æ –¥–ª—è –ø–æ–¥—Ç–≤–µ—Ä–∂–¥–µ–Ω...        145.0   
3  –í –Ω–∞—Å—Ç–æ—è—â–µ–µ –≤—Ä–µ–º—è –¥–æ—Å—Ç—É–ø–Ω–∞ —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è —Ç–æ–ª—å–∫–æ ...        145.0   
4  –í—ã–±–æ—Ä —è–∑—ã–∫–∞ –∑–∞–≤–∏—Å–∏—Ç –æ—Ç –Ω–∞—Å—Ç—Ä–æ–µ–∫ –±—Ä–∞—É–∑–µ—Ä–∞.  Chr...        145.0   

   page_char_count  page_word_count  page_sentence_count_raw  page_token_count  
0            335.0             45.0                      8.0             83.75  
1            138.0             22.0                      3.0             34.50  
2            431.0             60.0                      8.0            107.75  
3             67.0              9.0                      2.0             16.75  
4            718.0            108.0                      9.0            179.50

In [ ]:
cols = df_new.columns.tolist()
cols = cols[1:] + [cols[0]]
df_new = df_new[cols]

df_new.head()

page_number  page_char_count  page_word_count  page_sentence_count_raw  \
0        145.0            335.0             45.0                      8.0   
1        145.0            138.0             22.0                      3.0   
2        145.0            431.0             60.0                      8.0   
3        145.0             67.0              9.0                      2.0   
4        145.0            718.0            108.0                      9.0   

   page_token_count                                               text  
0             83.75  1. –î–ª—è —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏–∏ –≤ –ª–∏—á–Ω–æ–º –∫–∞–±–∏–Ω–µ—Ç–µ –ø–µ—Ä–µ–π–¥–∏—Ç–µ...  
1             34.50  –ö–Ω–æ–ø–∫–∞ —Å —Ñ–ª–∞–≥–æ–º –≤ –ø—Ä–∞–≤–æ–º –≤–µ—Ä—Ö–Ω–µ–º —É–≥–ª—É –Ω—É–∂–Ω–∞ –¥–ª...  
2            107.75  I. –ï—Å–ª–∏ –í–∞–º –Ω–µ –ø—Ä–∏—Ö–æ–¥–∏—Ç –ø–∏—Å—å–º–æ –¥–ª—è –ø–æ–¥—Ç–≤–µ—Ä–∂–¥–µ–Ω...  
3             16.75  –í –Ω–∞—Å—Ç–æ—è—â–µ–µ –≤—Ä–µ–º—è –¥–æ—Å—Ç—É–ø–Ω–∞ —Ä–µ–≥–∏—Å—Ç—Ä–∞—Ü–∏—è —Ç–æ–ª—å–∫–æ ...  
4            179.50  –í—ã–±–æ—Ä —è–∑—ã–∫–∞ –∑–∞–≤–∏—Å–∏—Ç –æ—Ç –Ω–∞—Å—Ç—Ä–æ–µ–∫ –±—Ä–∞—É–∑–µ—Ä–∞.  Chr...

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.head()

page_number  page_char_count  page_word_count  page_sentence_count_raw  \
0            0               35                7                        1   
1            1              198               23                        2   
2            3               45                6                        1   
3            4               57                8                        1   
4            6               49                6                        1   

   page_token_count                                               text  
0              8.75                –° —á–µ–≥–æ –Ω–∞—á–∞—Ç—å –ø—Ä–∏ —Ä–∞–±–æ—Ç–µ —Å –°–∏—Å—Ç–µ–º–æ–π  
1             49.50  –í—ã –Ω–∞—á–∏–Ω–∞–µ—Ç–µ —Ä–∞–±–æ—Ç—É —Å –°–∏—Å—Ç–µ–º–æ–π GreenData, –∫–æ—Ç–æ...  
2             11.25      –±—ã—Å—Ç—Ä–µ–µ –≤–Ω–µ–¥—Ä—è—Ç—å –Ω—É–∂–Ω—ã–µ —Å–µ—Ä–≤–∏—Å—ã –∏ –ø—Ä–∏–ª–æ–∂–µ–Ω–∏—è,  
3             14.25  –Ω–µ —Å–æ–≥–ª–∞—Å–æ–≤—ã–≤–∞—è —Å—Ç–µ–∫ —Ç–µ—Ö–Ω–æ–ª–æ–≥–∏–π –∏ –ø—Ä–∞–≤–∏–ª–∞ –æ–±–º–µ...  
4             12.25  –≤–Ω–æ—Å–∏—Ç—å –ª—é–±—ã–µ –∏–∑–º–µ–Ω–µ–Ω–∏—è –≤ –∏–Ω—Ñ–æ—Ä–º–∞—Ü–∏–æ–Ω–Ω—É—é —Å–∏—Å—Ç–µ–º—É,

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   page_number              2014 non-null   int64  
 1   page_char_count          2014 non-null   int64  
 2   page_word_count          2014 non-null   int64  
 3   page_sentence_count_raw  2014 non-null   int64  
 4   page_token_count         2014 non-null   float64
 5   text                     2014 non-null   object 
dtypes: float64(1), int64(4), object(1)
memory usage: 94.5+ KB


In [ ]:
df = pd.concat([df, df_new], ignore_index=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2307 entries, 0 to 2306
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   page_number              2307 non-null   float64
 1   page_char_count          2307 non-null   float64
 2   page_word_count          2307 non-null   float64
 3   page_sentence_count_raw  2307 non-null   float64
 4   page_token_count         2307 non-null   float64
 5   text                     2307 non-null   object 
dtypes: float64(5), object(1)
memory usage: 108.3+ KB


In [ ]:
# –ü—Ä–µ–æ–±—Ä–∞–∑–æ–≤–∞–Ω–∏–µ –¥–∞—Ç–∞—Ñ—Ä–µ–π–º–∞ –≤ —Å–ª–æ–≤–∞—Ä—å
pages_and_texts = df.to_dict(orient='records')

In [ ]:
pages_and_texts

[{'page_number': 0.0,
  'page_char_count': 35.0,
  'page_word_count': 7.0,
  'page_sentence_count_raw': 1.0,
  'page_token_count': 8.75,
  'text': '–° —á–µ–≥–æ –Ω–∞—á–∞—Ç—å –ø—Ä–∏ —Ä–∞–±–æ—Ç–µ —Å –°–∏—Å—Ç–µ–º–æ–π'},
 {'page_number': 1.0,
  'page_char_count': 198.0,
  'page_word_count': 23.0,
  'page_sentence_count_raw': 2.0,
  'page_token_count': 49.5,
  'text': '–í—ã –Ω–∞—á–∏–Ω–∞–µ—Ç–µ —Ä–∞–±–æ—Ç—É —Å –°–∏—Å—Ç–µ–º–æ–π GreenData, –∫–æ—Ç–æ—Ä–∞—è —Å–æ–¥–µ—Ä–∂–∏—Ç –ø–µ—Ä–µ–¥–æ–≤–æ–π —Ñ—É–Ω–∫—Ü–∏–æ–Ω–∞–ª –∞–≤—Ç–æ–º–∞—Ç–∏–∑–∞—Ü–∏–∏ –∏ —Å–ø–æ—Å–æ–±–Ω–∞ –ø—Ä–µ–æ–±—Ä–∞–∑–∏—Ç—å —Ä–∞–±–æ—Ç—É –ª—é–±–æ–≥–æ –±–∏–∑–Ω–µ—Å–∞. –ë–ª–∞–≥–æ–¥–∞—Ä—è –ø–µ—Ä–µ–¥–æ–≤—ã–º –∏–Ω—Å—Ç—Ä—É–º–µ–Ω—Ç–∞–º –∞–≤—Ç–æ–º–∞—Ç–∏–∑–∞—Ü–∏–∏ –í—ã —Å–º–æ–∂–µ—Ç–µ:'},
 {'page_number': 3.0,
  'page_char_count': 45.0,
  'page_word_count': 6.0,
  'page_sentence_count_raw': 1.0,
  'page_token_count': 11.25,
  'text': '–±—ã—Å—Ç—Ä–µ–µ –≤–Ω–µ–¥—Ä—è—Ç—å –Ω—É–∂–Ω—ã–µ —Å–µ—Ä–≤–∏—Å—ã –∏ –ø—Ä–∏–ª–æ–∂–µ–Ω–∏—è,'},
 {'page_number': 4.0,
  'page_c

In [ ]:
# Download NLTK's punkt package, used for tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Loop through each page and text extracted from the Word document
for item in tqdm(pages_and_texts):
    # Tokenize text into sentences using NLTK
    item["sentences"] = sent_tokenize(item["text"])

    # Ensure all sentences are represented as strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the number of sentences
    item["page_sentence_count_nltk"] = len(item["sentences"])

  0%|          | 0/2307 [00:00<?, ?it/s]

In [ ]:
# Create a DataFrame to get stats
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

page_number  page_char_count  page_word_count  page_sentence_count_raw  \
count      2307.00          2307.00          2307.00                  2307.00   
mean       1635.12            95.09            12.16                     1.52   
std        1156.29           114.69            14.83                     1.42   
min           0.00             1.00             1.00                     1.00   
25%         514.00            19.00             2.00                     1.00   
50%        1586.00            58.00             7.00                     1.00   
75%        2637.00           131.00            17.00                     1.00   
max        3729.00          1615.00           208.00                    19.00   

       page_token_count  page_sentence_count_nltk  
count           2307.00                   2307.00  
mean              23.77                      1.41  
std               28.67                      1.19  
min                0.25                      1.00  
25%                4.75                      1.00  
50%               14.50                      1.00  
75%               32.75                      1.00  
max              403.75                     16.00

In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/2307 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=1)

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

In [ ]:

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/2307 [00:00<?, ?it/s]

2312

In [ ]:
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': 541.0,
  'sentence_chunk': '–≠–∫–∑–µ–º–ø–ª—è—Ä ‚Äì –Ω–∞–±–æ—Ä —Å–≤–µ–¥–µ–Ω–∏–π –æ –∫–æ–Ω–∫—Ä–µ—Ç–Ω–æ–π —Å—É—â–Ω–æ—Å—Ç–∏;',
  'chunk_char_count': 49,
  'chunk_word_count': 7,
  'chunk_token_count': 12.25}]

In [ ]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

page_number  chunk_char_count  chunk_word_count  chunk_token_count
count      2312.00           2312.00           2312.00            2312.00
mean       1631.90             94.35             11.77              23.59
std        1157.11            111.23             14.12              27.81
min           0.00              1.00              1.00               0.25
25%         507.50             19.00              2.00               4.75
50%        1581.00             58.00              7.00              14.50
75%        2634.50            131.25             16.00              32.81
max        3729.00           1319.00            181.00             329.75

In [ ]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 12.0 | Text: DeletePlainObjects ‚Äì —É–¥–∞–ª–∏—Ç—å –ø—Ä–∏–º–∏—Ç–∏–≤–Ω—ã–π –æ–±—ä–µ–∫—Ç.
Chunk token count: 7.0 | Text: –≤—ã–π—Ç–∏ ‚Äì –≤—ã—Ö–æ–¥ –∏–∑ –ø—Ä–æ—Ñ–∏–ª—è –õ–ö;
Chunk token count: 29.75 | Text: –ø—Ä–æ–µ–∫—Ç—ã –∏ —É—á–∞—Å—Ç–Ω–∏–∫–∏ ‚Äì –ø—Ä–∏–≥–ª–∞—à–µ–Ω–∏–µ –∫–æ–ª–ª–µ–≥ –¥–ª—è —Å–æ–≤–º–µ—Å—Ç–Ω–æ–π —Ä–∞–±–æ—Ç—ã –Ω–∞–¥ –ø—Ä–æ–µ–∫—Ç–æ–º (–ø–æ–¥—Ä–æ–±–Ω–µ–µ –≤ —Å—Ç–∞—Ç—å–µ "–ü—Ä–æ–µ–∫—Ç—ã –∏ —É—á–∞—Å—Ç–Ω–∏–∫–∏");
Chunk token count: 10.75 | Text: –º–æ–¥–µ—Ä–Ω–∏–∑–∏—Ä–æ–≤–∞—Ç—å —Ä–∞–∑–º–µ—Ç–∫—É —á–∞—Å—Ç–µ–π –¥–æ–∫—É–º–µ–Ω—Ç–æ–≤;
Chunk token count: 3.0 | Text: –î–ª–∏—Ç–µ–ª—å–Ω–æ—Å—Ç—å


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 1.0,
  'sentence_chunk': '–í—ã –Ω–∞—á–∏–Ω–∞–µ—Ç–µ —Ä–∞–±–æ—Ç—É —Å –°–∏—Å—Ç–µ–º–æ–π GreenData, –∫–æ—Ç–æ—Ä–∞—è —Å–æ–¥–µ—Ä–∂–∏—Ç –ø–µ—Ä–µ–¥–æ–≤–æ–π —Ñ—É–Ω–∫—Ü–∏–æ–Ω–∞–ª –∞–≤—Ç–æ–º–∞—Ç–∏–∑–∞—Ü–∏–∏ –∏ —Å–ø–æ—Å–æ–±–Ω–∞ –ø—Ä–µ–æ–±—Ä–∞–∑–∏—Ç—å —Ä–∞–±–æ—Ç—É –ª—é–±–æ–≥–æ –±–∏–∑–Ω–µ—Å–∞.–ë–ª–∞–≥–æ–¥–∞—Ä—è –ø–µ—Ä–µ–¥–æ–≤—ã–º –∏–Ω—Å—Ç—Ä—É–º–µ–Ω—Ç–∞–º –∞–≤—Ç–æ–º–∞—Ç–∏–∑–∞—Ü–∏–∏ –í—ã —Å–º–æ–∂–µ—Ç–µ:',
  'chunk_char_count': 197,
  'chunk_word_count': 22,
  'chunk_token_count': 49.25},
 {'page_number': 13.0,
  'sentence_chunk': '–ü–æ—Å—Ç–µ–ø–µ–Ω–Ω–æ –í—ã –Ω–∞—É—á–∏—Ç–µ—Å—å —ç—Ñ—Ñ–µ–∫—Ç–∏–≤–Ω–æ –ø—Ä–∏–º–µ–Ω—è—Ç—å –≤–µ—Å—å –∏–Ω—Å—Ç—Ä—É–º–µ–Ω—Ç–∞—Ä–∏–π –ø–ª–∞—Ç—Ñ–æ—Ä–º—ã GreenData, –∞ —Å–µ–π—á–∞—Å –æ–∑–Ω–∞–∫–æ–º—å—Ç–µ—Å—å —Å –ø—Ä–µ–¥–Ω–∞–∑–Ω–∞—á–µ–Ω–∏–µ–º –æ—Å–Ω–æ–≤–Ω—ã—Ö –º–æ–¥—É–ª–µ–π.',
  'chunk_char_count': 143,
  'chunk_word_count': 16,
  'chunk_token_count': 35.75}]

In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="intfloat/multilingual-e5-large",
                                      device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
%%time

# Send the model to the GPU
embedding_model.to(device) # requires a GPU installed, for reference on my local machine, I'm using a NVIDIA RTX 4090

# Create embeddings one by one on the GPU
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/653 [00:00<?, ?it/s]

CPU times: user 10min 24s, sys: 1.26 s, total: 10min 25s
Wall time: 10min 45s


In [ ]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [ ]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, # you can use different batch sizes here for speed/performance, I found 32 works well for this use case
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

CPU times: user 9min 9s, sys: 52.2 s, total: 10min 1s
Wall time: 10min 8s


tensor([[ 0.0065, -0.0175, -0.0427,  ..., -0.0061, -0.0342,  0.0180],
        [ 0.0051, -0.0067, -0.0319,  ...,  0.0105, -0.0333,  0.0213],
        [ 0.0333, -0.0372, -0.0437,  ...,  0.0020, -0.0133,  0.0128],
        ...,
        [ 0.0097, -0.0399, -0.0107,  ..., -0.0167, -0.0113, -0.0009],
        [-0.0080, -0.0200, -0.0224,  ..., -0.0054, -0.0262,  0.0054],
        [ 0.0103, -0.0436, -0.0328,  ..., -0.0111, -0.0182,  0.0079]])

In [ ]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
%cd '/content/drive/MyDrive/–ú–§–¢–ò/–≤–∫—Ä/RAG'

/content/drive/MyDrive/–ú–§–¢–ò/–≤–∫—Ä/RAG


In [ ]:
current_directory = os.getcwd()
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = os.path.join(current_directory, 'DATA', 'text_chunks_and_embeddings_df2.csv')
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

page_number                                     sentence_chunk  \
0          1.0  –í—ã –Ω–∞—á–∏–Ω–∞–µ—Ç–µ —Ä–∞–±–æ—Ç—É —Å –°–∏—Å—Ç–µ–º–æ–π GreenData, –∫–æ—Ç–æ...   
1         13.0  –ü–æ—Å—Ç–µ–ø–µ–Ω–Ω–æ –í—ã –Ω–∞—É—á–∏—Ç–µ—Å—å —ç—Ñ—Ñ–µ–∫—Ç–∏–≤–Ω–æ –ø—Ä–∏–º–µ–Ω—è—Ç—å –≤...   
2         18.0  –ú–æ–¥—É–ª—å "–ú–æ–¥–µ–ª—å –¥–∞–Ω–Ω—ã—Ö" –ø–æ–∑–≤–æ–ª—è–µ—Ç –æ—Ä–≥–∞–Ω–∏–∑–æ–≤–∞—Ç—å ...   
3         38.0  –í–æ–∑–º–æ–∂–Ω–æ—Å—Ç–∏ –º–æ–¥—É–ª—è "–ë–∏–∑–Ω–µ—Å-–ø—Ä–æ—Ü–µ—Å—Å—ã" –ø–æ–∑–≤–æ–ª—è—é—Ç...   
4         60.0  –í –ø–ª–∞—Ç—Ñ–æ—Ä–º–µ GreenData –ª–æ–≥–∏–∫–∞ —Ä–∞–±–æ—Ç—ã —Å—Ç—Ä–æ–∏—Ç—Å—è –Ω...   

   chunk_char_count  chunk_word_count  chunk_token_count  \
0               197                22              49.25   
1               143                16              35.75   
2               168                20              42.00   
3               148                16              37.00   
4               203                24              50.75   

                                           embedding  
0  [ 0.00653521 -0.01750944 -0.04272387 ... -0.00...  
1  [ 0.00511181 -0.0066931  -0.03192068 ...  0.01...  
2  [ 0.03332577 -0.03724583 -0.04366744 ...  0.00...  
3  [ 0.00963955 -0.03917069 -0.04323367 ... -0.00...  
4  [ 0.01142762 -0.04457758 -0.03888189 ...  0.00...